# <center> **Titanic**

# **Libraries**

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


import functions
import importlib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


import warnings


importlib.reload(functions)

<module 'functions' from 'c:\\Users\\Dell\\Documents\\AI\\Titanic\\Notebooks\\functions.py'>

# **Data Overview and Preprocessing**

In [52]:
data = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Titanic\Data\data.csv",
    index_col=False
)

train = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Titanic\Data\Data\train.csv",
    index_col=False
)

test = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Titanic\Data\Data\test.csv",
    index_col=False
)


random_state = 101
target = 'Transported'

# **Machine Learning**

## **Split Data Back to Train and Test**

In [53]:
train=data[data['PassengerId'].isin(train['PassengerId'].values)].copy()
test=data[data['PassengerId'].isin(test['PassengerId'].values)].copy()
test.drop(columns=target, inplace=True)

## **Train Test Split**

In [54]:
X = train.drop(target, axis=1)
y = train[target]

X, y = shuffle(X, y, random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

# **Drop Unneeded Features**

In [57]:
X.drop(['PassengerId', 'Group', 'CabinNumber'], axis=1, inplace=True)
test.drop(['PassengerId', 'Group', 'CabinNumber'], axis=1, inplace=True)

# **Log Transform**

The logarithm transform is used to decrease skew in distributions, especially with large outliers. It can make it easier for algorithms to 'learn' the correct relationships. We will apply it to the expenditure features as these are heavily skewed by outliers.

In [58]:
X['RoomService'] = np.log1p(X['RoomService'])
test['RoomService'] = np.log1p(test['RoomService'])

X['FoodCourt'] = np.log1p(X['FoodCourt'])
test['FoodCourt'] = np.log1p(test['FoodCourt'])

X['ShoppingMall'] = np.log1p(X['ShoppingMall'])
test['ShoppingMall'] = np.log1p(test['ShoppingMall'])

X['Spa'] = np.log1p(X['Spa'])   
test['Spa'] = np.log1p(test['Spa'])

X['VRDeck'] = np.log1p(X['VRDeck'])
test['VRDeck'] = np.log1p(test['VRDeck'])

X['Spent'] = np.log1p(X['Spent'])
test['Spent'] = np.log1p(test['Spent'])


In [60]:
X.dtypes

GroupSize         int64
Solo              int64
HomePlanet       object
CabinDeck        object
CabinSide        object
CryoSleep          bool
Destination      object
Age             float64
AgeGroup         object
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
TotalSpent      float64
Spent           float64
LastName         object
FamilySize        int64
LoneTraveler      int64
dtype: object

# **Encoding and Scaling**

In [62]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Indentify numerical and categorical columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in X.columns if X[cname].dtype == ["object", 'bool']]

# Scale numerical data to have mean=0 and variance=1
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# One-hot encode categorical data
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore'))])

# Combine preprocessing
ct = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)],
        remainder='passthrough')

# Apply preprocessing
train = ct.fit_transform(train)
test = ct.transform(test)

# Print new shape
print('Training set shape:', train.shape)

ValueError: columns are missing: {'PassengerId', 'Transported', 'Group', 'CabinNumber'}